<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [46]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
#sys.path.append("../FinRL-Library")

import itertools
import os

import pickle

<a id='2'></a>
# Part 3. Get Data


In [14]:
input_data_path = '../data'
input_data_filename = 'binance_BTCUSDT_1m_from_2020_01_01_to_2021_12_31_candlesticks_signals_processed'
input_data_extension = ".csv"
full_path_input_data = os.path.join(input_data_path, input_data_filename + input_data_extension)

In [15]:
df = pd.read_csv(full_path_input_data)

In [16]:
df.shape

(1048956, 38)

In [17]:
df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,CDLINVERTEDHAMMER_NEW,CDLHAMMER_NEW,CDLPIERCING_NEW,CDLMORNINGSTAR_NEW,CDLENGULFINGBULLISH_NEW,CDLSHOOTINGSTAR_NEW,CDLHANGINGMAN_NEW,CDLDARKCLOUDCOVER_NEW,CDLEVENINGSTAR_NEW,CDLENGULFINGBEARISH_NEW
0,1577836800000,7195.24,7196.25,7183.14,7186.68,51.642812,1577836859999,371233.518355,493,19.598230,...,0,0,0,0,0,0,0,0,0,0
1,1577836860000,7187.67,7188.06,7182.20,7184.03,7.248148,1577836919999,52080.127788,135,2.031772,...,0,0,0,0,0,0,0,0,0,0
2,1577836920000,7184.41,7184.71,7180.26,7182.43,11.681677,1577836979999,83903.741635,202,5.479244,...,0,0,0,0,0,0,0,0,0,0
3,1577836980000,7183.83,7188.94,7182.49,7185.94,10.025391,1577837039999,72033.226649,136,3.294966,...,0,0,0,0,0,0,0,0,0,0
4,1577837040000,7185.54,7185.54,7178.64,7179.78,14.911105,1577837099999,107066.521825,161,2.369033,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volumne', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore',
       'formatted_open_time', 'formatted_close_time', 'upper_shadow',
       'lower_shadow', 'real_body', 'CDLINVERTEDHAMMER', 'CDLHAMMER',
       'CDLPIERCING', 'CDLMORNINGSTAR', 'CDLSHOOTINGSTAR', 'CDLHANGINGMAN',
       'CDLDARKCLOUDCOVER', 'CDLEVENINGSTAR', 'CDLENGULFING',
       'CDLENGULFINGBULLISH', 'CDLENGULFINGBEARISH', 'CDLINVERTEDHAMMER_NEW',
       'CDLHAMMER_NEW', 'CDLPIERCING_NEW', 'CDLMORNINGSTAR_NEW',
       'CDLENGULFINGBULLISH_NEW', 'CDLSHOOTINGSTAR_NEW', 'CDLHANGINGMAN_NEW',
       'CDLDARKCLOUDCOVER_NEW', 'CDLEVENINGSTAR_NEW',
       'CDLENGULFINGBEARISH_NEW'],
      dtype='object')

In [19]:
df.rename(columns = {'formatted_open_time':'date'}, inplace = True)

In [20]:
df["tic"] = 'BTCUSDT'

In [21]:
df_col_selected = df[['date', 'tic', 'open', 'high', 'low', 'close', 'volume',
        'upper_shadow', 'lower_shadow', 'real_body', 
        'CDLINVERTEDHAMMER_NEW', 'CDLHAMMER_NEW', 'CDLMORNINGSTAR_NEW', 'CDLENGULFINGBULLISH_NEW', 
        'CDLSHOOTINGSTAR_NEW', 'CDLHANGINGMAN_NEW', 'CDLEVENINGSTAR_NEW', 'CDLENGULFINGBEARISH_NEW']]

In [22]:
df_col_selected.head()

,date,tic,open,high,low,close,volume,upper_shadow,lower_shadow,real_body,CDLINVERTEDHAMMER_NEW,CDLHAMMER_NEW,CDLMORNINGSTAR_NEW,CDLENGULFINGBULLISH_NEW,CDLSHOOTINGSTAR_NEW,CDLHANGINGMAN_NEW,CDLEVENINGSTAR_NEW,CDLENGULFINGBEARISH_NEW
0,2020-01-01 00:00:00,BTCUSDT,7195.24,7196.25,7183.14,7186.68,51.642812,1.01,3.54,8.56,0,0,0,0,0,0,0,0
1,2020-01-01 00:01:00,BTCUSDT,7187.67,7188.06,7182.20,7184.03,7.248148,0.39,1.83,3.64,0,0,0,0,0,0,0,0
2,2020-01-01 00:02:00,BTCUSDT,7184.41,7184.71,7180.26,7182.43,11.681677,0.30,2.17,1.98,0,0,0,0,0,0,0,0
3,2020-01-01 00:03:00,BTCUSDT,7183.83,7188.94,7182.49,7185.94,10.025391,3.00,1.34,2.11,0,0,0,0,0,0,0,0
4,2020-01-01 00:04:00,BTCUSDT,7185.54,7185.54,7178.64,7179.78,14.911105,0.00,1.14,5.76,0,0,0,0,0,0,0,0


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [25]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = ["macd", "boll_ub", "boll_lb", "rsi_30", "cci_30", "dx_30", "close_30_sma", "close_60_sma"],
                    use_vix=False,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df_col_selected)

/opt/homebrew/Caskroom/miniforge/base/envs/msc_uol_ai_dissertation/lib/python3.8/site-packages/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/opt/homebrew/Caskroom/miniforge/base/envs/msc_uol_ai_dissertation/lib/python3.8/site-packages/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/opt/homebrew/Caskroom/miniforge/base/envs/msc_uol_ai_dissertation/lib/python3.8/site-packages/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/opt/homebrew/Caskroom/miniforge/base/envs/msc

Successfully added technical indicators


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split:
## Trade data split:

In [26]:
train = data_split(processed, '2020-01-01','2021-06-30')
trade = data_split(processed, '2021-06-30','2021-12-31')
print(len(train))
print(len(trade))

784385
264570


In [27]:
train.tail()

,date,tic,open,high,low,close,volume,upper_shadow,lower_shadow,real_body,...,CDLEVENINGSTAR_NEW,CDLENGULFINGBEARISH_NEW,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
784380,2021-06-29 23:55:00,BTCUSDT,35858.08,35858.09,35830.36,35850.37,41.739582,0.01,20.01,7.71,...,0,0,-41.231184,36110.212018,35790.596982,41.757816,-131.336690,50.030617,35979.158333,35965.902500
784381,2021-06-29 23:56:00,BTCUSDT,35850.38,35925.29,35850.37,35914.00,58.950362,11.29,0.01,63.62,...,0,0,-36.693908,36088.377477,35795.031523,46.574800,-76.648525,14.307052,35977.260667,35964.135667
784382,2021-06-29 23:57:00,BTCUSDT,35914.00,35931.94,35900.30,35931.93,16.280007,0.01,13.70,17.93,...,0,0,-31.290586,36065.778232,35802.837768,47.832506,-51.123894,11.498641,35977.024667,35962.999833
784383,2021-06-29 23:58:00,BTCUSDT,35931.88,35935.00,35844.85,35857.84,26.735525,3.12,12.99,74.04,...,0,0,-32.610938,36048.284201,35801.060799,43.459132,-84.403884,27.398003,35973.028000,35960.910167
784384,2021-06-29 23:59:00,BTCUSDT,35857.84,35928.92,35830.12,35911.73,70.776395,17.19,27.72,53.89,...,0,0,-28.974851,36027.632456,35808.288544,47.098557,-69.425668,30.921991,35970.085667,35960.689000


In [28]:
trade.head()

,date,tic,open,high,low,close,volume,upper_shadow,lower_shadow,real_body,...,CDLEVENINGSTAR_NEW,CDLENGULFINGBEARISH_NEW,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2021-06-30 00:00:00,BTCUSDT,35911.72,35941.75,35868.84,35924.04,60.951928,17.71,42.88,12.32,...,0,0,-24.813872,36016.914824,35811.410176,47.891159,-47.501402,25.261039,35965.887000,35959.781500
1,2021-06-30 00:01:00,BTCUSDT,35924.03,35971.25,35850.00,35901.60,49.714898,47.22,51.60,22.43,...,0,0,-23.061153,36008.122908,35812.277092,46.575236,-47.674618,13.227638,35960.226667,35959.701833
2,2021-06-30 00:02:00,BTCUSDT,35901.60,35921.43,35830.00,35869.61,80.065702,19.83,39.61,31.99,...,0,0,-23.977043,35990.932415,35815.557585,44.761418,-74.272915,18.875646,35954.054333,35959.528667
3,2021-06-30 00:03:00,BTCUSDT,35867.91,35892.49,35830.00,35861.47,56.834397,24.58,31.47,6.44,...,0,0,-25.070723,35975.283518,35818.057482,44.307219,-81.920150,18.875646,35946.216000,35958.743000
4,2021-06-30 00:04:00,BTCUSDT,35861.44,35920.29,35844.72,35847.26,41.738493,58.85,2.54,14.18,...,0,0,-26.775450,35960.503204,35819.385796,43.509915,-70.129566,7.737266,35937.452667,35956.125667


In [29]:
list_indicators = ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']

In [30]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(list_indicators)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 1, State Space: 11


In [33]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount" : 1000,
    "num_stock_shares": stock_dimension,
    # buy and sell cost for each stock
    "buy_cost_pct": [0.001] * stock_dimension,
    "sell_cost_pct": [0.001] * stock_dimension,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": list_indicators, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [34]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [35]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [36]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [37]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 28        |
|    iterations         | 100       |
|    time_elapsed       | 17        |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.5      |
|    explained_variance | -1.23     |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -4.56e-05 |
|    reward             | 0.0       |
|    std                | 1.08      |
|    value_loss         | 7.65e-09  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 28        |
|    iterations         | 200       |
|    time_elapsed       | 34        |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -1.57     |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updat

In [50]:
# full_path_export_model_a2c = '../models/finrl_stocktrading_neurips_2018_mod_a2c.pkl'

# # Open a file and use dump()
# with open(full_path_export_model_a2c, 'wb') as file:
      
#     # A new file will be created
#     pickle.dump(trained_a2c, file, protocol=pickle.HIGHEST_PROTOCOL)

### Model 2: DDPG

In [38]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [39]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

In [51]:
# full_path_export_model_ddpg = '../models/finrl_stocktrading_neurips_2018_mod_ddpg.pkl'

# # Open a file and use dump()
# with open(full_path_export_model_ddpg, 'wb') as file:
      
#     # A new file will be created
#     pickle.dump(trained_ddpg, file)

### Model 3: PPO

In [40]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [41]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

-----------------------------
| time/              |      |
|    fps             | 21   |
|    iterations      | 1    |
|    time_elapsed    | 95   |
|    total_timesteps | 2048 |
| train/             |      |
|    reward          | 0.0  |
-----------------------------
--------------------------------------------
| time/                   |                |
|    fps                  | 18             |
|    iterations           | 2              |
|    time_elapsed         | 217            |
|    total_timesteps      | 4096           |
| train/                  |                |
|    approx_kl            | 0.0008110906   |
|    clip_fraction        | 0.000391       |
|    clip_range           | 0.2            |
|    entropy_loss         | -1.42          |
|    explained_variance   | -1.19e-07      |
|    learning_rate        | 0.00025        |
|    loss                 | -0.0134        |
|    n_updates            | 10             |
|    policy_gradient_loss | 3.75e-05       |
|    rewar

### Model 4: TD3

In [42]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [43]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

### Model 5: SAC

In [44]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 1000000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [45]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=60000)

## Trading
Assume that we have $1,000 initial capital at 2021-06-30. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_risk_indicator = processed[(processed.date<'2021-07-01') & (processed.date>='2020-01-01')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [52]:
# e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [53]:
trade.head()

,date,tic,open,high,low,close,volume,upper_shadow,lower_shadow,real_body,...,CDLEVENINGSTAR_NEW,CDLENGULFINGBEARISH_NEW,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2021-06-30 00:00:00,BTCUSDT,35911.72,35941.75,35868.84,35924.04,60.951928,17.71,42.88,12.32,...,0,0,-24.813872,36016.914824,35811.410176,47.891159,-47.501402,25.261039,35965.887000,35959.781500
1,2021-06-30 00:01:00,BTCUSDT,35924.03,35971.25,35850.00,35901.60,49.714898,47.22,51.60,22.43,...,0,0,-23.061153,36008.122908,35812.277092,46.575236,-47.674618,13.227638,35960.226667,35959.701833
2,2021-06-30 00:02:00,BTCUSDT,35901.60,35921.43,35830.00,35869.61,80.065702,19.83,39.61,31.99,...,0,0,-23.977043,35990.932415,35815.557585,44.761418,-74.272915,18.875646,35954.054333,35959.528667
3,2021-06-30 00:03:00,BTCUSDT,35867.91,35892.49,35830.00,35861.47,56.834397,24.58,31.47,6.44,...,0,0,-25.070723,35975.283518,35818.057482,44.307219,-81.920150,18.875646,35946.216000,35958.743000
4,2021-06-30 00:04:00,BTCUSDT,35861.44,35920.29,35844.72,35847.26,41.738493,58.85,2.54,14.18,...,0,0,-26.775450,35960.503204,35819.385796,43.509915,-70.129566,7.737266,35937.452667,35956.125667


In [54]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


In [55]:
df_account_value.shape

(264570, 2)

In [61]:
df_account_value.head()

,date,account_value
0,2021-06-30 00:00:00,36924.04
1,2021-06-30 00:01:00,1000.00
2,2021-06-30 00:02:00,1000.00
3,2021-06-30 00:03:00,1000.00
4,2021-06-30 00:04:00,1000.00


In [ ]:
df_account_value.tail()

,date,account_value
264565,2021-12-30 23:55:00,1000.0
264566,2021-12-30 23:56:00,1000.0
264567,2021-12-30 23:57:00,1000.0
264568,2021-12-30 23:58:00,1000.0
264569,2021-12-30 23:59:00,1000.0


In [57]:
df_actions.head()

,date,actions
0,2021-06-30 00:00:00,[0]
1,2021-06-30 00:01:00,[0]
2,2021-06-30 00:02:00,[0]
3,2021-06-30 00:03:00,[0]
4,2021-06-30 00:04:00,[0]


In [67]:
df_actions.actions

0         [0]
1         [0]
2         [0]
3         [0]
4         [0]
         ... 
264564    [0]
264565    [0]
264566    [0]
264567    [0]
264568    [0]
Name: actions, Length: 264569, dtype: object

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [58]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("results/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return         -0.003432
Cumulative returns    -0.972917
Annual volatility      0.030027
Sharpe ratio          -0.030862
Calmar ratio          -0.003527
Stability              0.000000
Max drawdown          -0.972917
Omega ratio            0.000000
Sortino ratio         -0.030862
Skew                        NaN
Kurtosis                    NaN
Tail ratio                  NaN
Daily value at risk   -0.003787
dtype: float64


/opt/homebrew/Caskroom/miniforge/base/envs/msc_uol_ai_dissertation/lib/python3.8/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/opt/homebrew/Caskroom/miniforge/base/envs/msc_uol_ai_dissertation/lib/python3.8/site-packages/empyrical/stats.py:1527: RuntimeWarning: invalid value encountered in double_scalars
  return np.abs(np.percentile(returns, 95)) / \


In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])